In [ ]:
%load_ext autoreload
%autoreload 2

import sys; sys.path.append("../")

import cv2
from src.lib.cv import normalize_dicom
from pathlib import Path
from pydicom import dcmread
import plotly.express as px
import numpy as np


In [ ]:
DICOM_PATH = Path("../data/train_images/426/824075419.dcm")

In [ ]:
dicom = dcmread(DICOM_PATH)
image = normalize_dicom(dicom)

if dicom.ImageLaterality == "R":
    image = cv2.flip(image, 1)

image = cv2.resize(image, dsize=None, fx=0.2, fy=0.2, interpolation=cv2.INTER_AREA)
fig = px.imshow(image)
fig.show(renderer="png")

In [ ]:
image *= 255
image = image.astype(np.uint8)

In [ ]:
threshold, threshed_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
fig = px.imshow(threshed_image)
fig.show(renderer="png")

In [ ]:
edges = cv2.Canny(image, 25, 50)
fig = px.imshow(edges)
fig.show(renderer="png")

In [ ]:
binary_image = cv2.add(threshed_image, edges)
fig = px.imshow(binary_image)
fig.show(renderer="png")

In [ ]:
closing = cv2.morphologyEx(binary_image, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7)), iterations=5)
fig = px.imshow(closing)
fig.show(renderer="png")

In [ ]:
opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7)), iterations=3)
fig = px.imshow(opening)
fig.show(renderer="png")

In [ ]:
nb_components, output, stats, centroid = cv2.connectedComponentsWithStats(closing)

In [ ]:
areas = [label_stats[cv2.CC_STAT_AREA] for label_stats in stats[1:]]
largest_component_index = areas.index(max(areas)) + 1

In [ ]:
largest_component_mask = output == largest_component_index

In [ ]:
fig = px.imshow(largest_component_mask.astype(np.float64))
fig.show(renderer="png")

In [ ]:
cc_left = stats[largest_component_index, cv2.CC_STAT_LEFT]
cc_top = stats[largest_component_index, cv2.CC_STAT_TOP]
cc_width = stats[largest_component_index, cv2.CC_STAT_WIDTH]
cc_height = stats[largest_component_index, cv2.CC_STAT_HEIGHT]

largest_component_mask = largest_component_mask.astype(np.float64)

In [ ]:
breast_cut = largest_component_mask[cc_top: cc_top + cc_height, cc_left: cc_left + cc_width]
fig = px.imshow(breast_cut)
fig.show(renderer="png")

In [ ]:
image_cut = image[cc_top: cc_top + cc_height, cc_left: cc_left + cc_width]
fig = px.imshow(image_cut)
fig.show(renderer="png")